## Initial pre-processing

In this notebook we will:
- Proceed with basic cleaning of each year from **2015-2020** which includes removing nan speakers, nan quotes and further inconsistencies 
- Exploit the useful information from the wikidata dumps provided to us 
- Add columns of interest for further analysis 

The **output** will be pickle files for each year with additional columns such as tags, gender (male/female), domain name, citizenship of spokesperson...
This output file will be used in the **second notebook** containing the `basic data analysis`


#### Useful libraries

In [1]:
import pandas as pd
import pickle
from tqdm.auto import trange, tqdm
import time
from journal_API_wikidata import extract_info_wiki
from Data_clean_functions import *
from tld import get_tld

from collections import Counter
import warnings
warnings.filterwarnings("ignore")

 

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/lavinia/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/lavinia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
# Whether to run cleaning or not
RUN_CLEANING = False
# Note: Approximate 4 hours per year --> over a day on a single computer

### Data paths

In [4]:
"""
    Please note the data is not provided but the `FILE` represents the 
    data given by QuoteBank 
"""

DATA_PATH = './data/'

# Original files from Quotebank
FILES = [DATA_PATH + 'quotes-2015.json.bz2', DATA_PATH + 'quotes-2016.json.bz2', DATA_PATH + 'quotes-2017.json.bz2',
         DATA_PATH + 'quotes-2018.json.bz2', DATA_PATH + 'quotes-2019.json.bz2', DATA_PATH + 'quotes-2020.json.bz2']

# Files after rapid cleaning and explose function 
# Note: Because of the explose function we get a great number of lines, thus the saved file is large in size (approx 4 times original)
PATHS_OUT =[ DATA_PATH + 'rapid_clean-quotes-2015.json.bz2',DATA_PATH + 'rapid_clean-quotes-2016.json.bz2',
            DATA_PATH + 'rapid_clean-quotes-2017.json.bz2',DATA_PATH + 'rapid_clean-quotes-2018.json.bz2',
            DATA_PATH + 'rapid_clean-quotes-2019.json.bz2',DATA_PATH + 'rapid_clean-quotes-2020.json.bz2',]

PATHS_OUT_filter = [DATA_PATH + 'filter_clean-quotes-2015.json.bz2',DATA_PATH + 'filter_clean-quotes-2016.json.bz2',
                   DATA_PATH + 'filter_clean-quotes-2017.json.bz2',DATA_PATH + 'filter_clean-quotes-2018.json.bz2',
                   DATA_PATH + 'filter_clean-quotes-2019.json.bz2',DATA_PATH + 'filter_clean-quotes-2020.json.bz2']

#### Read file, clean and save in pickle

In [5]:
def clean_original_file_top_sites(FILE, PATH_OUT):
    
    with pd.read_json(FILE, lines=True, compression='bz2', chunksize=100000) as df_reader:
        for chunk in tqdm(df_reader):

            # Basic cleaning (refer to function doc)
            df_base_clean = rapid_clean(chunk)

            # Extract site name from dataframe
            extract_name(df_base_clean)

            # Expand the df on sitenames and urls
            df_base_clean_exp = df_base_clean.explode(["sitenames", "urls"])

            # Save chunk by chunk appending the clean df
            with open(PATH_OUT, 'ab') as d_file:
                pickle.dump(df_base_clean_exp, d_file)
                n_chunks += 1

            # Add counter for occurences of a specific media
            counts = Counter(df_base_clean_exp['sitenames'].tolist()) 
            Total_count += counts
            print("Chunk done")

        # List the top 100 most occuring media
        for site, count in Total_count.most_common(100):
                top_sites.append(site)
        
        
        # Pickle save the top_sites for future use
        with open(DATA_PATH + f'top_sites_{FILE[14:18]}.pkl', 'wb') as output:
            pickle.dump(top_sites, output)
        TOP_LIST.append(top_sites)

        print("finished top sites")
    
    
    return n_chunks, TOP_LSIT



# Save the number of chunks dumped for each file in order to open again later
N_CHUNKS_LIST = []

# Save the top sites for quick access in following function
TOP_LIST = []

if RUN_CLEANING:
    for index, FILE in enumerate(FILES):

        n_chunks = 0
        Total_count = Counter()
        top_sites = []

        n_chunks,TOP_LIST = clean_original_file_top_sites(FILE, PATHS_OUT[index], TOP_LIST)
        N_CHUNKS_LIST.append(n_chunks)
    


In [6]:

# Pickle open the top_sites
"""
with open(DATA_PATH + 'top_sites.pkl', 'rb') as file:
    top_sites = pickle.load(file)
    
"""

"\nwith open(DATA_PATH + 'top_sites.pkl', 'rb') as file:\n    top_sites = pickle.load(file)\n    \n"

### For Milestone 2 keep 10 most citing media


**Note**: We intend on increasing the number from 10 to 50 but this would take 
an approximate 5 hours to run per year, we thus plan on running it after the deadline 
and focused on making a main pipeline first



### Filter the rows belonging to top 10 sites

In [9]:
# New df with rows belonging to top 10 sites
#PATHS_OUT_filter
def extract_top_sites_rows(PATH_OUT, top_10_sites, n_chunks, PATH_OUT_filter):
    
    chunks_all_filtered = pd.DataFrame(columns=['quoteID', 'quotation', 'speaker', 'qids', 'date', 'numOccurrences',
           'probas', 'urls', 'phase', 'sitenames'])

    chunk_nbr = 0

    with open(PATH_OUT, 'rb') as d_file:
        while (chunk_nbr < n_chunks):

            # Progress meter
            print(f"{chunk_nbr}/{n_chunks}")

            chunk = pickle.load(d_file)

            # Filter chunks with sitenames belonging to top 10
            chunk_filtered = chunk[chunk.sitenames.isin(top_10_sites)]

            # Save filtered chunks
            with open(PATH_OUT_filter, 'ab') as d_file_out:
                pickle.dump(chunk_filtered, d_file_out)


            chunks_all_filtered = chunks_all_filtered.append(chunk_filtered)

            chunk_nbr+=1

        # Save as pickle for future use
        with open(DATA_PATH + f'chunks_all_filtered_{FILE[14:18]}.pkl', 'wb') as output:
            pickle.dump(chunks_all_filtered, output)
if RUN_CLEANING:
    for index, PATH_OUT in enumerate(PATHS_OUT):

        # TOP_SITES[index][:10] are the first top 10 sites for a year
        extract_top_sites_rows(PATH_OUT, TOP_SITES[index][:10], N_CHUNKS_LIST[index], PATHS_OUT_filter[index])

**Note:**

The function above may be used to process all yearly quotes but this requires a powerful computer and a lot of memory (as mentionned with the exploded dataset). Thus we each treated different years for efficiency.

Below we will pursue the last part of the analysis with an example on one specific year. For eg:2020
This can be and was done for each pre-processed year (2015-2020)

In [10]:


if RUN_CLEANING:
    # Open pickled dataframe
    with open(DATA_PATH +'chunks_all_filtered_2020.pkl', 'rb') as output:
        chunks_all_filtered = pickle.load(output)



In [11]:
if RUN_CLEANING:
    # Groupby the exploded data set 
    gb_all_filtered = chunks_all_filtered[["speaker", "qids" , "urls", "quoteID", "quotation","date"]].groupby(["speaker", "qids", "quoteID", "quotation"])

    # One row, quote, may be cited by different media so we list them
    df_filtered = gb_all_filtered["urls"].apply(list)

    df_filtered_final = df_filtered.reset_index()

In [12]:

if RUN_CLEANING:
    # Save pickled dataframe
    with open(DATA_PATH + 'df_filtered_final_2020.pkl', 'wb') as output:
        pickle.dump(df_filtered_final, output)

    '''# Open pickled dataframe
    with open(DATA_PATH + 'df_filtered_final.pkl', 'rb') as output:
        df_filtered = pickle.load(output)
    '''

### Create a dictionnary of categories and associated synonyms

This will enable us to tag the different category of the quote

*Note*: This is a restrictive list and some additional content will be added

In [13]:
if RUN_CLEANING:
    matchers = {"art": ["art", "paint", "draw", "museum"], \
                "business": ["business", "finance", "economy", "commerce", "bank", "money", "trade"], \
                "entertainment":["entertainment"], 
                "fashion":["fashion", "couture", "designer"], \
                "medicine":["medicine", "health", "pharmacy", "wellbeing", "body"], \
                "music":["music", "song", "album", "concert"], \
                "politics":["politics", "government"], \
                "science":["science", "research"], \
                "sport": ["sport", "football", "athletics", "swimming", "rugby", "tennis", "volleyball", "ski"]}

    # Find general form for categories and words
    generalizeDictionary(matchers)

### Extract information from URL

In [14]:
if RUN_CLEANING:
    df_extract = Chunk_url_extract(df_filtered_final, matchers)


In [15]:
if RUN_CLEANING:
    with open(DATA_PATH + 'df_extract_2020.pkl', 'wb') as output:
        pickle.dump(df_extract, output)

In [16]:
"""
# Open pickled dataframe
with open(DATA_PATH + 'df_extract.pkl', 'rb') as output:
    df_extract = pickle.load(output)
"""

"\n# Open pickled dataframe\nwith open(DATA_PATH + 'df_extract.pkl', 'rb') as output:\n    df_extract = pickle.load(output)\n"

### Formatting wikidata data of interest
Using the Wikidata speakers and label description files provided by TA's, we extract data we need for our project.

This includes gender, citizenship, data of birth...

**Note**:The file is the same for each year so it was saved and utilized by each of us for the respective year we treated

In [17]:
if RUN_CLEANING:
    Wikidata_speakers = pd.read_parquet(DATA_PATH + 'speaker_attributes.parquet')
    Wikidata_countries = pd.read_csv(DATA_PATH + 'wikidata_labels_descriptions_quotebank.csv.bz2', compression = 'bz2')

    Wikidata_utils = formating_wikidata(Wikidata_speakers, Wikidata_countries)

In [18]:
if RUN_CLEANING:
    # Save wikidata utils
    with open(DATA_PATH + 'Wikidata_utils.pkl', 'wb') as output:
        pickle.dump(Wikidata_utils, output)

In [19]:
if RUN_CLEANING:
    # Open file 
    with open(DATA_PATH + 'Wikidata_utils.pkl', 'rb') as input_file:
        Wikidata_utils = pickle.load(input_file)

### Add information from wikidata

In [20]:
if RUN_CLEANING:
    # Merge the info of wikidata from speaker to the initial dataframe
    df_merged = merge_quotes_wikidata(Wikidata_utils, df_extract)

### Final dataframe after merging 

The final dataframe for each year after the basic cleaning and merging is given in the drive link provided in the README.MD


**Note**: These data frames are the ones used in the Notebook on the "Basic data analysis "

In [21]:
if RUN_CLEANING:
    with open(DATA_PATH + 'df_2020_no_media.pkl', 'wb') as output:
        pickle.dump(df_merged, output)